# The Cozo database tutorial

This tutorial will teach you the basics of using the Cozo database.

The best way to learn from this tutorial is to run the queries as they are introduced.
For this, you need to install Cozo on your local machine.

Cozo is distributed as a single gzipped/zipped binary. 
Go to https://github.com/cozodb/cozo/releases and download the latest release for your operating system to a local directory.

If your operating system is Linux/Mac, 
open a terminal command prompt, 
`cd` into the directory of your download and run

```bash
gunzip cozoserver-*.gz
mv cozoserver-* cozoserver
chmod +x cozoserver
```

If you are on Windows instead, open a PowerShell and run

```powershell
Expand-Archive -Path .\cozoserver-*.zip -DestinationPath .
```

To run the server, you need to specify a directory to store persistent data on your file system. 
In the following, we will use a directory called `tutorial-data` in the same directory as the binary executable.
Run:

```bash
./cozoserver ./tutorial-data
```

The same command should work in PowerShell as well.

If you see something like `Database web API running at ...` displayed in your terminal, 
then the server is successfully started. 
Keep the server running when you are following the rest of the tutorial.
When you are done, `CTRL-C` will stop the server.

## A place to run queries

Cozo exposes an HTTP API, so theoretically you can follow along using tools like `curl`, as described [here](https://cozodb.github.io/current/manual/setup.html#making-http-requests).
But for a better user experience, we suggest following one of the following two subsections instead.

### Option 1: the JupyterLab notebook

This option provides the best user experience but requires you to install [Python](https://www.python.org/downloads/) and some other stuff.
After you have Python installed, install [JupyterLab](https://jupyter.org/install).
Next, run the following in a terminal to install a Jupyter extension to help query Cozo:

```bash
pip install pycozo pandas
```

While you are at it, right-click on https://raw.githubusercontent.com/cozodb/cozo/main/docs/tutorial/tutorial.ipynb 
and save the notebook of this tutorial to your disk.

Then run Jupyter Lab and open the saved tutorial document.

We need to enable the extension in the notebook. Run

In [1]:
%load_ext pycozo.ipyext_direct

Then run the "hello world" query to verify everything works:

In [2]:
?[] <- [['hello', 'world', 'Cozo!']]

,0,1,2
0,hello,world,Cozo!


If you get the same words back formatted in a table, congratulations! 
You can skip to the next section where we start learning CozoScript proper.

### Option 2: the JavaScript console in your browser

If you have never used Python before, the first option may be overwhelming. 

Whatever your reason for not wanting to install the whole python toolchain, 
if your local machine has a modern web browser such as a recent version of Firefox, Chrome, or Edge, 
you can use this option.

Use your browser to navigate to http://127.0.0.1:9070 (the address shown in your terminal when you run `cozoserver`).
You should be greeted by a page saying that the server is running.
Now open the developer tools of your browser by right-clicking the page and selecting "Inspect" from the menu 
(if you cannot find it, you may need to fiddle with your browser settings to enable the developer tools).
Switch to the "Console" tab of the developer tools if it is not already open. 

If you see some messages where 
the `Cozo Makeshift Javascript Console` welcomes you, you are ready. Run the "hello world" query by typing the following into the console and pressing enter:

```javascript
await run(`?[] <- [['hello', 'world', 'Cozo!']]`)
```

If you see the three words echoed back in a table, you are successful. When following the tutorial, you have to wrap all queries within the backticks `` in the above command to run them in the JavaScript console.

## Your first relations

Cozo is a relational database. The "hello world" query

In [3]:
?[] <- [['hello', 'world', 'Cozo!']]

,0,1,2
0,hello,world,Cozo!


simply passes an ad hoc relation represented by a list of lists to the database, and the database echoes it back.

You can pass more rows, or a different number of columns:

In [4]:
?[] <- [[1, 2, 3], ['a', 'b', 'c']]

,0,1,2
0,1,2,3
1,a,b,c


This example shows how to enter literals for numbers, strings, booleans and `null`:

In [5]:
?[] <- [[1.5, 2.5, 3, 4, 5.5], 
        ['aA', 'bB', 'cC', 'dD', 'eE'], 
        [true, false, null, -1.4e-2, "A string with double quotes"]]

,0,1,2,3,4
0,True,False,None,-0.014000,A string with double quotes
1,1.500000,2.500000,3,4,5.500000
2,aA,bB,cC,dD,eE


The input literal representations are similar to those in JavaScript. 
In particular, strings in double quotes are guaranteed to be interpreted in the same way as in JSON.
The output are in JSON, but how they appear on your screen depends on your setup.
For example, if you are using a Python setup, booleans are displayed as `True` and `False` instead of in lowercase,
since they are converted to the native Python datatypes for display.

In the last example, you may have noticed that the returned order is not the same as the input order.
This is because in Cozo relations are always stored as trees, and trees are always sorted.

Another consequence of relations as trees is that you can have no duplicate rows:

In [6]:
?[] <- [[1], [2], [1], [2], [1]]

,0
0,1
1,2


Relations in Cozo follow _set semantics_ where de-duplication is automatic. 
By contrast, SQL usually follows _bag semantics_ (some databases do this by secretly having a unique internal key for every row, in Cozo you must do this explicitly if you need to simulate duplicate rows).

Why does Cozo break tradition and go with set semantics?
Set semantics is much more convenient when you have recursions between relations involved,
and Cozo is designed to deal with complex recursions.

## Expressions

The next example shows the use of various expressions and comments:

In [7]:
?[] <- [[
            1 + 2, # addition
            3 / 4, # division
            5 == 6, # equality
            7 > 8, # greater
            true || false, # or
            false && true, # and
            lowercase('HELLO'), # function
            rand_float(), # function taking no argument
            union([1, 2, 3], [3, 4, 5], [5, 6, 7]), # variadic function
        ]]

,0,1,2,3,4,5,6,7,8
0,3,0.750000,False,False,True,False,hello,0.998594,"[1, 2, 3, 4, 5, 6, 7]"


See [here](https://cozodb.github.io/current/manual/functions.html) for the full list of functions you can use to build expressions.
The syntax is deliberately C-like.

## Rules and relations

Previous examples all start with `?[] <-`.
The notation denotes a _rule_ named `?`. It is a _constant rule_ since it uses `<-`.
The part after `?` and before `<-` is the _head_ of the rule, and the part after `<-` is the _body_.

Rules can have other names, but the rule named `?` is special in that its evaluation determines the return relation of the query.

We can give _bindings_ in the _head_ of rules:

In [8]:
?[first, second, third] <- [[1, 2, 3], ['a', 'b', 'c']]

,first,second,third
0,1,2,3
1,a,b,c


For constant rules, the number of bindings must match the actual data (the _arity_), otherwise, you will get an error:

In [9]:
?[first, second] <- [[1, 2, 3], ['a', 'b', 'c']]

parser::fixed_rule_head_arity_mismatch

  × Fixed rule head arity mismatch
   ╭────
 1 │ ?[first, second] <- [[1, 2, 3], ['a', 'b', 'c']]
   · ─────────────────────────────────────────────────
   ╰────
  help: Expected arity: 3, number of arguments given: 2


Now let's define rules that _apply_ (use) other rules:

In [10]:
rule[first, second, third] <- [[1, 2, 3], ['a', 'b', 'c']]
?[a, b, c] := rule[a, b, c]

,a,b,c
0,1,2,3
1,a,b,c


This first line defines a constant rule named `rule`. The `?` rule is now an _inline rule_, denoted by the connecting symbol `:=`. In its body it applies the fixed rule, by giving the name of the rule followed by three _fresh bindings_, which are the _variables_ `a`, `b` and `c`.

With inline rules, you can manipulate the order of the columns, or specify which columns are returned:

In [11]:
rule[first, second, third] <- [[1, 2, 3], ['a', 'b', 'c']]
?[c, b] := rule[a, b, c]

,c,b
0,3,2
1,c,b


The body of an inline rule consists of _atoms_. 
The previous example has a single _application atom_ as the body. Multiple atoms are connected by commas:

In [12]:
?[c, b] := rule[a, b, c], is_num(a)
rule[first, second, third] <- [[1, 2, 3], ['a', 'b', 'c']]

,c,b
0,3,2


Here the second atom is an _expression_ `is_num(a)`. 
Expression atoms act as filters, so only rows for which the expression evaluates to `true` are returned. 
The order in which the rules are given is immaterial.

You can also bind constants to rule applications directly:

In [13]:
rule[first, second, third] <- [[1, 2, 3], ['a', 'b', 'c']]
?[c, b] := rule['a', b, c]

,c,b
0,c,b


You introduce additional bindings with the _unification operator_ `=`:

In [14]:
rule[first, second, third] <- [[1, 2, 3], ['a', 'b', 'c']]
?[c, b, d] := rule[a, b, c], is_num(a), d = a + b + 2*c

,c,b,d
0,3,2,9


Having multiple rule applications in the body generates every combination of the bindings:

In [15]:
r1[] <- [[1, 'a'], [2, 'b']]
r2[] <- [[2, 'B'], [3, 'C']]

?[l1, l2] := r1[a, l1], 
             r2[b, l2]

,l1,l2
0,a,B
1,a,C
2,b,B
3,b,C


This is a Cartesian join in relational algebra, as bindings in the rule applications are all distinct.
If bindings are reused, _implicit unification_ occurs, 
which is a join in relational algebra:

In [16]:
r1[] <- [[1, 'a'], [2, 'b']]
r2[] <- [[2, 'B'], [3, 'C']]

?[l1, l2] := r1[a, l1], 
             r2[a, l2] # reused `a`

,l1,l2
0,b,B


The unification `=` unifies with a single value. Use `in` to unify with each value within a list in turn:

In [17]:
?[x, y] := x in [1, 2, 3], y in ['x', 'y']

,x,y
0,1,x
1,1,y
2,2,x
3,2,y
4,3,x
5,3,y


The head of inline rules does not need to use all variables that appear in the body. 
However, any variable in the head must appear in the body at least once (the _safety rule_):

In [18]:
r1[] <- [[1, 'a'], [2, 'b']]
r2[] <- [[2, 'B'], [3, 'C']]

?[l1, l2, x] := r1[a, l1], 
                r2[a, l2]

eval::unbound_symb_in_head

  × Symbol 'x' in rule head is unbound
   ╭─[3:1]
 3 │ 
 4 │ ?[l1, l2, x] := r1[a, l1], 
   ·           ─
 5 │                 r2[a, l2]
   ╰────
  help: Note that symbols occurring only in negated positions are not considered bound


## Stored relations

Persistent relations in Cozo are called _stored relations_.
Creating them are simple:

In [19]:
r1[] <- [[1, 'a'], [2, 'b']]
r2[] <- [[2, 'B'], [3, 'C']]

?[l1, l2] := r1[a, l1], 
             r2[a, l2]
    
:create stored {l1, l2}

,status
0,OK


The `:create` query option instructs the database to store the result in a stored relation named `stored`, containing the columns `l1` and `l2`.

If you just want to create the relation without adding any data, you can omit the queries. No need to have an empty `?` query.

You can verify that you now have the required stored relation in your system by running a _system op_:

In [20]:
::relations

,name,arity,access_level,n_keys,n_non_keys,n_put_triggers,n_rm_triggers,n_replace_triggers
0,stored,2,normal,2,0,0,0,0


You can also investigate the columns of the stored relation:

In [21]:
::columns stored 

,column,is_key,index,type,has_default
0,l1,True,0,Any?,False
1,l2,True,1,Any?,False


Stored relations can be applied by using an asterisk `*` before the name:

In [22]:
?[a, b] := *stored[a, b]

,a,b
0,b,B


Unlike relations defined inline, the columns of stored relations have fixed names. 
You can take advantage of this by selectively referring to columns by name,
especially if you have a lot of columns:

In [23]:
?[a, b] := *stored{l2: b, l1: a}

,a,b
0,b,B


If the binding is the same as the column name, you can omit the binding:

In [24]:
?[l2] := *stored{l2}

,l2
0,B


After a stored relation is created, use `:put` to insert more data:

In [25]:
?[l1, l2] <- [['e', 'E']]
    
:put stored {l1, l2}

,status
0,OK


In [26]:
?[l1, l2] := *stored[l1, l2]

,l1,l2
0,b,B
1,e,E


Use `:rm` to remove data:

In [27]:
?[l1, l2] <- [['e', 'E']]
    
:rm stored {l1, l2}

,status
0,OK


In [28]:
?[l1, l2] := *stored[l1, l2]

,l1,l2
0,b,B


Use `::remove` (double colon!) to get rid of a stored relation:

In [29]:
::remove stored

,status
0,OK


In [30]:
::relations

,name,arity,access_level,n_keys,n_non_keys,n_put_triggers,n_rm_triggers,n_replace_triggers


So far, all stored relations store all the data in their _keys_.
You can instruct Cozo to only treat some of the data as keys, thereby indicating a _functional dependency_:

In [31]:
?[a, b, c] <- [[1, 'a', 'A'],
               [2, 'b', 'B'],
               [3, 'c', 'C'],
               [4, 'd', 'D']]

:create fd {a, b => c}

,status
0,OK


In [32]:
?[a, b, c] := *fd[a, b, c]

,a,b,c
0,1,a,A
1,2,b,B
2,3,c,C
3,4,d,D


Now if you insert another row with an existing key, that row will be updated:

In [33]:
?[a, b, c] <- [[3, 'c', 'CCCCCCC']]

:put fd {a, b => c}

,status
0,OK


In [34]:
?[a, b, c] := *fd[a, b, c]

,a,b,c
0,1,a,A
1,2,b,B
2,3,c,CCCCCCC
3,4,d,D


You can check whether a column is in a key position by looking at the `is_key` column in:

In [35]:
::columns fd

,column,is_key,index,type,has_default
0,a,True,0,Any?,False
1,b,True,1,Any?,False
2,c,False,2,Any?,False


You may have noticed that columns also have types and default values associated with them, and stored relations can have triggers. These are discussed in the [manual](https://cozodb.github.io/current/manual/stored.html).
We won't overload you with all the complexities in this tutorial.

Before continuing, let's remove the stored relation we introduced:

In [36]:
::remove fd

,status
0,OK


## Graphs

Now let's consider a graph stored as a relation:

In [37]:
?[loving, loved] <- [['alice', 'eve'],
                     ['bob', 'alice'],
                     ['eve', 'alice'],
                     ['eve', 'bob'],
                     ['eve', 'charlie'],
                     ['charlie', 'eve'],
                     ['david', 'george'],
                     ['george', 'george']]

:replace love {loving, loved}

,status
0,OK


The graph we have created reads like "Alice loves Eve, Bob loves Alice", "nobody loves David, David loves George, but George only loves himself", and so on. 
Here we used `:replace` instead of `:create`. The difference is that if `love` already exists, it will be wiped and replaced with the new data given.

We can investigate competing interests:

In [38]:
?[loved_by_b_e] := *love['eve', loved_by_b_e], *love['bob', loved_by_b_e]

,loved_by_b_e
0,alice


So far we rule bodies consist of _conjunction_ of atoms only. Disjunction is also available, by using the `or` keyword:

In [39]:
?[loved_by_b_e] := *love['eve', loved_by_b_e] or *love['bob', loved_by_b_e], 
                   loved_by_b_e != 'bob', 
                   loved_by_b_e != 'eve'

,loved_by_b_e
0,alice
1,charlie


Another way to write the same query is to have multiple rule definitions under the same name:

In [40]:
?[loved_by_b_e] := *love['eve', loved_by_b_e], 
                   loved_by_b_e != 'bob', 
                   loved_by_b_e != 'eve'
?[loved_by_b_e] := *love['bob', loved_by_b_e], 
                   loved_by_b_e != 'bob', 
                   loved_by_b_e != 'eve'

,loved_by_b_e
0,alice
1,charlie


When you have multiple definitions of the same inline rule, the rule heads must be compatible. Only inline rules can have multiple definitions.

## Negation

Negation of _expressions_ should be familiar:

In [41]:
?[loved] := *love[person, loved], !ends_with(person, 'e')

,loved
0,alice
1,george


Rule applications can also be negated, not with the `!` operator, but with the `not` keyword:

In [42]:
?[loved_by_e_not_b] := *love['eve', loved_by_e_not_b], not *love['bob', loved_by_e_not_b]

,loved_by_e_not_b
0,bob
1,charlie


There are two sets of logical operations in Cozo, one set that acts on the level of expressions, and another set that acts on the level of atoms:

* For atoms: `,` or `and` (conjunction), `or` (disjunction), `not` (negation)
* For expressions: `&&` (conjunction), `||` (disjunction), `!` (negation)

The difference between `,` and `and` is operator precedence: `and` has higher precedence than `or`, whereas `,` has lower precedence than `or`.

There is a _safety rule_ for negation:

In [43]:
?[not_loved_by_b] := not *love['bob', not_loved_by_b]

eval::unbound_symb_in_head

  × Symbol 'not_loved_by_b' in rule head is unbound
   ╭────
 1 │ ?[not_loved_by_b] := not *love['bob', not_loved_by_b]
   ·   ──────────────
   ╰────
  help: Note that symbols occurring only in negated positions are not considered bound


This query is forbidden because the resulting relation is infinite.
For example, 'gold' should be in the result, since _according to the facts stored in the database_, 
Bob has no interest in 'gold'.

To make our query finite, we have to explicitly give our query a _closed world_:

In [44]:
the_population[p] := *love[p, _a]
the_population[p] := *love[_a, p]

?[not_loved_by_b] := the_population[not_loved_by_b], not *love['bob', not_loved_by_b]

,not_loved_by_b
0,bob
1,charlie
2,david
3,eve
4,george


## Recursion

Inline rules can applying other rules, and can have multiple definitions. If you combine these two, you get recursions:

In [45]:
alice_love_chain[person] := *love['alice', person]
alice_love_chain[person] := alice_love_chain[in_person], *love[in_person, person]

?[chained] := alice_love_chain[chained]

,chained
0,alice
1,bob
2,charlie
3,eve


You may object that you only need to be able to apply other rules to have recursion, 
without multiple definitions. 
Technically correct, but the resulting queries are not useful::

In [46]:
alice_love_chain[person] := alice_love_chain[in_person], *love[in_person, person]

?[chained] := alice_love_chain[chained]

,chained


Similar to the negation case, if there is no way to _deduce_ a fact from the given facts, 
then the fact itself is considered false. 
You need multiple definitions to "bootstrap" the query.

## Aggregation

_Aggregations_ are usually used to compute statistics. In Cozo, aggregations are applied in the head of inline rules:

In [47]:
?[person, count(loved_by)] := *love[loved_by, person]

,person,count(loved_by)
0,alice,2
1,bob,1
2,charlie,1
3,eve,2
4,george,2


The usual `sum`, `mean`, etc. are all available. 
Aggregations in the head instead of in the body may seem strange, but is powerful, as we will see later.

Here is the [full list](https://cozodb.github.io/current/manual/aggregations.html) of aggregations for you to play with.

## Query options

We have seem query options like `:create`, `:put`, `:rm` for manipulating stored relations. 
There are also query options for controlling what is returned:

In [48]:
?[loving, loved] := *love{ loving, loved }

:limit 1

,loving,loved
0,alice,eve


Next we want the result to be sorted by `loved` in descending order, then `loving` in ascending order, and skip the first row:

In [49]:
?[loving, loved] := *love{ loving, loved }

:order -loved, loving
:offset 1

,loving,loved
0,george,george
1,alice,eve
2,charlie,eve
3,eve,charlie
4,eve,bob
5,bob,alice
6,eve,alice


Putting `-` in front of variables in `:order` clause denotes reverse order. Nothing or `+` denotes the ascending order.

The full list of query options are explained [here](https://cozodb.github.io/current/manual/queries.html#query-options).

## Fixed rules

The `<-` syntax for constant rules is syntax sugar. The full syntax is:

In [50]:
?[] <~ Constant(data: [['hello', 'world', 'Cozo!']])

,0,1,2
0,hello,world,Cozo!


Here we are using the _fixed rule_ `Constant`, which takes one _option_ named `data`. The curly tail `<~` denotes a fixed rule.

Fixed rules take input relations as arguments, apply custom logic to them and produce its output relation. 
The `Constant` fixed rule take zero input relations.

As an example of another fixed rule, let's find out who is most popular in the `love` graph. 
How to define popularity? 
One way is to say that the higher [PageRank](https://en.wikipedia.org/wiki/PageRank) a person has, the more popular. 
Calculating PageRank using inline rules
is very awkward (but doable). Fortunately, one of the fixed rules is an optimized PageRank implementation:

In [51]:
?[person, page_rank] <~ PageRank(*love[])

:order -page_rank

,person,page_rank
0,alice,1.191497
1,eve,1.191497
2,george,1.064742
3,bob,0.921087
4,charlie,0.921087
5,david,0.574623


Here the input relation is the stored relation `*love`. 

Each fixed rule is different. Read [this](https://cozodb.github.io/current/manual/algorithms.html) to learn how to use them.

In [52]:
::remove love

,status
0,OK


## Extended example: the air routes dataset

Now you have a basic understanding of using the various constructs of Cozo, let's deal with a less trivial dataset.

The data we are going to use, and many examples that we will present, are adapted from the book [Practical Gremlin](https://kelvinlawrence.net/book/Gremlin-Graph-Guide.html). Gremlin is an imperative query language for graphs, a very different take compared to Datalog.

We will download the data directly from the Internet. First let us set up some _parameters_ to be used in queries:

In [53]:
%cozo_set AIR_ROUTES_NODES_URL 'https://raw.githubusercontent.com/cozodb/cozo/main/tests/air-routes-latest-nodes.csv'
%cozo_set AIR_ROUTES_EDGES_URL 'https://raw.githubusercontent.com/cozodb/cozo/main/tests/air-routes-latest-edges.csv'
# %cozo_set AIR_ROUTES_NODES_URL 'file://./tests/air-routes-latest-nodes.csv'
# %cozo_set AIR_ROUTES_EDGES_URL 'file://./tests/air-routes-latest-edges.csv'

If you want, you can manually download the files and refer to them locally. The commented lines show how to do it.

If you are not using Jupyter, pass parameters to queries in a map. For example, in the JavaScript console:

```js
await run(`<QUERY>`, {'AIR_ROUTES_NODES_URL': 'https://...', 'AIR_ROUTES_EDGES_URL': 'https://...'})
```

Let's import the data into our database. First, the `airport` relation:

In [54]:
res[idx, label, typ, code, icao, desc, region, runways, longest, elev, country, city, lat, lon] <~
    CsvReader(types: ['Int', 'Any', 'Any', 'Any', 'Any', 'Any', 'Any', 'Int?', 'Float?', 'Float?', 'Any', 'Any', 'Float?', 'Float?'],
              url: $AIR_ROUTES_NODES_URL, 
              has_headers: true)

?[code, icao, desc, region, runways, longest, elev, country, city, lat, lon] :=
    res[idx, label, typ, code, icao, desc, region, runways, longest, elev, country, city, lat, lon],
    label == 'airport'

:replace airport {
    code: String 
    => 
    icao: String, 
    desc: String, 
    region: String, 
    runways: Int, 
    longest: Float, 
    elev: Float, 
    country: String, 
    city: String, 
    lat: Float, 
    lon: Float
}

,status
0,OK


The `CsvReader` utility downloads a CSV file from the internet and attempts to parse its content into a relation.
When we store the relation, we specified types for the columns. The `code` column acts as a primary key for the `airport` stored relation.

Next is `country`:

In [55]:
res[idx, label, typ, code, icao, desc] <~
    CsvReader(types: ['Int', 'Any', 'Any', 'Any', 'Any', 'Any'],
              url: $AIR_ROUTES_NODES_URL, 
              has_headers: true)
?[code, desc] :=
    res[idx, label, typ, code, icao, desc],
    label == 'country'

:replace country {
    code: String
    =>
    desc: String
}

,status
0,OK


`continent`:

In [56]:
res[idx, label, typ, code, icao, desc] <~
    CsvReader(types: ['Int', 'Any', 'Any', 'Any', 'Any', 'Any'],
              url: $AIR_ROUTES_NODES_URL, 
              has_headers: true)
?[idx, code, desc] :=
    res[idx, label, typ, code, icao, desc],
    label == 'continent'

:replace continent {
    code: String
    =>
    desc: String
}

,status
0,OK


We need to make a translation table for the indices the data use:

In [57]:
res[idx, label, typ, code] <~
    CsvReader(types: ['Int', 'Any', 'Any', 'Any'],
              url: $AIR_ROUTES_NODES_URL, 
              has_headers: true)
?[idx, code] :=
    res[idx, label, typ, code],

:replace idx2code { idx => code }

,status
0,OK


The `contain` relation contains information on the geographical inclusion of entities:

In [58]:
res[] <~
    CsvReader(types: ['Int', 'Int', 'Int', 'String'],
              url: $AIR_ROUTES_EDGES_URL, 
              has_headers: true)
?[entity, contained] :=
    res[idx, fr_i, to_i, typ],
    typ == 'contains',
    *idx2code[fr_i, entity],
    *idx2code[to_i, contained]


:replace contain { entity: String, contained: String }

,status
0,OK


Finally, the `route`s between the airports. This relation is much larger than the rest and contains about 60k rows:

In [59]:
res[] <~
    CsvReader(types: ['Int', 'Int', 'Int', 'String', 'Float?'],
              url: $AIR_ROUTES_EDGES_URL, 
              has_headers: true)
?[fr, to, dist] :=
    res[idx, fr_i, to_i, typ, dist],
    typ == 'route',
    *idx2code[fr_i, fr],
    *idx2code[to_i, to]

:replace route { fr: String, to: String => dist: Float }

,status
0,OK


We no longer need the `idx2code` relation:

In [60]:
::remove idx2code

,status
0,OK


Let's verify all the relations we want are there:

In [61]:
::relations

,name,arity,access_level,n_keys,n_non_keys,n_put_triggers,n_rm_triggers,n_replace_triggers
0,airport,11,normal,1,10,0,0,0
1,contain,2,normal,2,0,0,0,0
2,continent,2,normal,1,1,0,0,0
3,country,2,normal,1,1,0,0,0
4,route,3,normal,2,1,0,0,0


While we are at it, let's lock all these tables to prevent accidentally changing their contents:

In [62]:
::access_level read_only airport, contain, continent, country, route

,status
0,OK


More information about what this does is explained [here](https://cozodb.github.io/current/manual/sysops.html#ops-for-stored-relations).

Let's just look at some data. Start with airports:

In [63]:
?[code, city, desc, region, runways, lat, lon] := *airport{code, city, desc, region, runways, lat, lon}
    
:limit 5

,code,city,desc,region,runways,lat,lon
0,AAA,Anaa,Anaa Airport,PF-U-A,1,-17.352600,-145.509995
1,AAE,Annabah,Annaba Airport,DZ-36,2,36.822201,7.809170
2,AAL,Aalborg,Aalborg Airport,DK-81,2,57.092759,9.849243
3,AAN,Al Ain,Al Ain International Airport,AE-AZ,1,24.261700,55.609200
4,AAQ,Anapa,Anapa Airport,RU-KDA,1,45.002102,37.347301


Airports with the most runways:

In [64]:
?[code, city, desc, region, runways, lat, lon] := *airport{code, city, desc, region, runways, lat, lon}

:order -runways
:limit 10

,code,city,desc,region,runways,lat,lon
0,DFW,Dallas,Dallas/Fort Worth International Airport,US-TX,7,32.896801,-97.038002
1,ORD,Chicago,Chicago O'Hare International Airport,US-IL,7,41.978600,-87.904800
2,AMS,Amsterdam,Amsterdam Airport Schiphol,NL-NH,6,52.308601,4.763890
3,BOS,Boston,Boston Logan,US-MA,6,42.364300,-71.005203
4,DEN,Denver,Denver International Airport,US-CO,6,39.861698,-104.672997
5,DTW,Detroit,"Detroit Metropolitan, Wayne County",US-MI,6,42.212399,-83.353401
6,ATL,Atlanta,Hartsfield - Jackson Atlanta International Airport,US-GA,5,33.636700,-84.428101
7,GIS,Gisborne,Gisborne Airport,NZ-GIS,5,-38.663300,177.977997
8,HLZ,Hamilton,Hamilton International Airport,NZ-WKO,5,-37.866699,175.332001
9,IAH,Houston,George Bush Intercontinental,US-TX,5,29.984400,-95.341400


How many airports are there in total?

In [65]:
?[count(code)] := *airport{code}

,count(code)
0,3504


Let's get a distribution of the initials of the airport codes:

In [66]:
?[count(initial), initial] := *airport{code}, initial = first(chars(code))

:order initial

,count(initial),initial
0,212,A
1,235,B
2,214,C
3,116,D
4,95,E
5,76,F
6,135,G
7,129,H
8,112,I
9,80,J


More useful are the statistics of runways:

In [67]:
?[count(r), count_unique(r), sum(r), min(r), max(r), mean(r), std_dev(r)] := 
    *airport{runways: r}

,count(r),count_unique(r),sum(r),min(r),max(r),mean(r),std_dev(r)
0,3504,7,4980.000000,1,7,1.421233,0.743083


Using `country`, we can find countries with no airports:

In [68]:
?[desc] := *country{code, desc}, not *airport{country: code}

,desc
0,Andorra
1,Liechtenstein
2,Monaco
3,Pitcairn
4,San Marino


The `route` relation by itself is rather boring:

In [69]:
?[fr, to, dist] := *route{fr, to, dist}

:limit 10

,fr,to,dist
0,AAA,FAC,48.000000
1,AAA,MKP,133.000000
2,AAA,PPT,270.000000
3,AAA,RAR,968.000000
4,AAE,ALG,254.000000
5,AAE,CDG,882.000000
6,AAE,IST,1161.000000
7,AAE,LYS,631.000000
8,AAE,MRS,477.000000
9,AAE,ORN,477.000000


It just records the starting and ending airports of each route, together with the distance. This relation only becomes useful when used as a graph.

Airports with no routes:

In [70]:
?[code, desc] := *airport{code, desc}, not *route{fr: code}, not *route{to: code}

,code,desc
0,AFW,Fort Worth Alliance Airport
1,APA,Centennial Airport
2,APK,Apataki Airport
3,BID,Block Island State Airport
4,BVS,Breves Airport
5,BWU,Sydney Bankstown Airport
6,CRC,Santa Ana Airport
7,CVT,Coventry Airport
8,EKA,Murray Field
9,GYZ,Gruyere Airport


Airports with the most out routes:

In [71]:
route_count[fr, count(fr)] := *route{fr}
?[code, n] := route_count[code, n]

:sort -n
:limit 5

,code,n
0,FRA,310
1,IST,309
2,CDG,293
3,AMS,283
4,MUC,270


How many routes are there from the European Union to the US?

In [72]:
routes[unique(r)] := *contain['EU', fr],
                     *route{fr, to},
                     *airport{code: to, country: 'US'},
                     r = [fr, to]
?[n] := routes[rs], n = length(rs)

,n
0,435


How many airports are there in the US with routes from the EU?

In [73]:
?[count_unique(to)] := *contain['EU', fr],
                       *route{fr, to},
                       *airport{code: to, country: 'US'}


,count_unique(to)
0,45


How many routes are there for each airport in London, UK?

In [74]:
?[code, count(code)] := *airport{code, city: 'London', region: 'GB-ENG'}, *route{fr: code}

,code,count(code)
0,LCY,51
1,LGW,232
2,LHR,221
3,LTN,130
4,STN,211


We need to specify the region, because there is another city called London, not in the UK.

How many airports are reachable from London, UK in two hops?

In [75]:
lon_uk_airports[code] := *airport{code, city: 'London', region: 'GB-ENG'}
one_hop[to] := lon_uk_airports[fr], *route{fr, to}, not lon_uk_airports[to];
?[count_unique(a3)] := one_hop[a2], *route{fr: a2, to: a3}, not lon_uk_airports[a3];

,count_unique(a3)
0,2353


What are the cities directly reachable from LGW (London Gatwick), but furthermost away?

In [76]:
?[city, dist] := *route{fr: 'LGW', to, dist},
                 *airport{code: to, city}
:order -dist
:limit 10

,city,dist
0,Buenos Aires,6908.000000
1,Singapore,6751.000000
2,Langkawi,6299.000000
3,Duong Dong,6264.000000
4,Taipei,6080.000000
5,Port Louis,6053.000000
6,Rayong,6008.000000
7,Cape Town,5987.000000
8,Hong Kong,5982.000000
9,Shanghai,5745.000000


What airports are within 0.1 degrees of the Greenwich meridian?

In [77]:
?[code, desc, lon, lat] := *airport{lon, lat, code, desc}, lon > -0.1, lon < 0.1

,code,desc,lon,lat
0,CDT,Castellon De La Plana Airport,0.026111,39.999199
1,LCY,London City Airport,0.055278,51.505278
2,LDE,Tarbes-Lourdes-Pyrénées Airport,-0.006439,43.178699
3,LEH,Le Havre Octeville Airport,0.088056,49.533901


Airports in a box drawn around London Heathrow, UK:

In [78]:
h_box[lon, lat] := *airport{code: 'LHR', lon, lat}
?[code, desc] := h_box[lhr_lon, lhr_lat], *airport{code, lon, lat, desc},
                 abs(lhr_lon - lon) < 1, abs(lhr_lat - lat) < 1

,code,desc
0,LCY,London City Airport
1,LGW,London Gatwick
2,LHR,London Heathrow
3,LTN,London Luton Airport
4,SOU,Southampton Airport
5,STN,London Stansted Airport


For some spherical geometry: what is the angle subtended by SFO and NRT on the surface of the earth?

In [79]:
?[deg_diff] := *airport{code: 'SFO', lat: a_lat, lon: a_lon},
               *airport{code: 'NRT', lat: b_lat, lon: b_lon},
               deg_diff = rad_to_deg(haversine_deg_input(a_lat, a_lon, b_lat, b_lon))

,deg_diff
0,73.992112


We mentioned before that aggregations in Cozo are powerful. They are powerful because they can be used in recursions (some restrictions apply).

Let's find the distance of the _shortest route_ between two airports. One wayis to enumerate all the routes between the two airports, and then apply `min` aggregation to the results. This cannot be implemented as stated, since the routes may contain cycles and hence there can be an infinite number of routes between two airports.

Instead, think recursively. If we already have all the shortest routes between all nodes, we derive an _equation_ satisfied by the shortest route: the shortest route between `a` and `b` is either the distance of a direct route, or the sum of the shortest distance from `a` to `c` and the distance of a direct route from `c` to `d`. We apply our `min` aggregation to this recursive set instead.

Write it out and it works. For exmaple, the shortest routes between the airports `LHR` and `YPO`:

In [80]:
shortest[b, min(dist)] := *route{fr: 'LHR', to: b, dist} 
                          # Start with the airport 'LHR', retrieve a direct route from 'LHR' to b

shortest[b, min(dist)] := shortest[c, d1], # Start with an existing shortest route from 'LHR' to c
                          *route{fr: c, to: b, dist: d2},  # Retrieve a direct route from c to b
                          dist = d1 + d2 # Add the distances

?[dist] := shortest['YPO', dist] # Extract the answer for 'YPO'. 
                                 # We chose it since it is the hardest airport to get to from 'LHR'.

,dist
0,4147.000000


Since path-finding is such a common operation on graphs, Cozo has several fixed rules for that:

In [81]:
starting[] <- [['LHR']]
goal[] <- [['YPO']]
?[starting, goal, distance, path] <~ ShortestPathDijkstra(*route[], starting[], goal[])

,starting,goal,distance,path
0,LHR,YPO,4147.000000,"['LHR', 'YUL', 'YVO', 'YKQ', 'YMO', 'YFA', 'ZKE', 'YAT', 'YPO']"


Not only is it more efficient, but we also get a path for the shortest route.

Not content with the shortest path, the following calculates ten the shortest paths:

In [82]:
starting[] <- [['LHR']]
goal[] <- [['YPO']]
?[starting, goal, distance, path] <~ KShortestPathYen(*route[], starting[], goal[], k: 10)

,starting,goal,distance,path
0,LHR,YPO,4147.000000,"['LHR', 'YUL', 'YVO', 'YKQ', 'YMO', 'YFA', 'ZKE', 'YAT', 'YPO']"
1,LHR,YPO,4150.000000,"['LHR', 'DUB', 'YUL', 'YVO', 'YKQ', 'YMO', 'YFA', 'ZKE', 'YAT', 'YPO']"
2,LHR,YPO,4164.000000,"['LHR', 'YUL', 'YMT', 'YKQ', 'YMO', 'YFA', 'ZKE', 'YAT', 'YPO']"
3,LHR,YPO,4167.000000,"['LHR', 'DUB', 'YUL', 'YMT', 'YKQ', 'YMO', 'YFA', 'ZKE', 'YAT', 'YPO']"
4,LHR,YPO,4187.000000,"['LHR', 'MAN', 'DUB', 'YUL', 'YVO', 'YKQ', 'YMO', 'YFA', 'ZKE', 'YAT', 'YPO']"
5,LHR,YPO,4202.000000,"['LHR', 'IOM', 'DUB', 'YUL', 'YVO', 'YKQ', 'YMO', 'YFA', 'ZKE', 'YAT', 'YPO']"
6,LHR,YPO,4204.000000,"['LHR', 'MAN', 'DUB', 'YUL', 'YMT', 'YKQ', 'YMO', 'YFA', 'ZKE', 'YAT', 'YPO']"
7,LHR,YPO,4209.000000,"['LHR', 'YUL', 'YMT', 'YNS', 'YKQ', 'YMO', 'YFA', 'ZKE', 'YAT', 'YPO']"
8,LHR,YPO,4211.000000,"['LHR', 'MAN', 'IOM', 'DUB', 'YUL', 'YVO', 'YKQ', 'YMO', 'YFA', 'ZKE', 'YAT', 'YPO']"
9,LHR,YPO,4212.000000,"['LHR', 'DUB', 'YUL', 'YMT', 'YNS', 'YKQ', 'YMO', 'YFA', 'ZKE', 'YAT', 'YPO']"


If efficiency is really important to you, you can use the A* algorithm with a good heuristic function:

In [83]:
code_lat_lon[code, lat, lon] := *airport{code, lat, lon}
starting[code, lat, lon] := code = 'LHR', *airport{code, lat, lon};
goal[code, lat, lon] := code = 'YPO', *airport{code, lat, lon};
?[] <~ ShortestPathAStar(*route[], 
                         code_lat_lon[node, lat1, lon1], 
                         starting[], 
                         goal[goal, lat2, lon2], 
                         heuristic: haversine_deg_input(lat1, lon1, lat2, lon2) * 3963);

,0,1,2,3
0,LHR,YPO,4147.000000,"['LHR', 'YUL', 'YVO', 'YKQ', 'YMO', 'YFA', 'ZKE', 'YAT', 'YPO']"


There's a lot more setup required in this case: we need to retrieve the latitudes and longitudes of airports and do processing on them first.
The number `3963` above is the radius of the earth in miles.

The most important airports, by PageRank:

In [84]:
rank[code, score] <~ PageRank(*route[a, b])
?[code, desc, score] := rank[code, score], *airport{code, desc}

:limit 10;
:order -score

,code,desc,score
0,FRA,Frankfurt am Main,1.265292
1,IST,Istanbul International Airport,1.260846
2,CDG,Paris Charles de Gaulle,1.251049
3,AMS,Amsterdam Airport Schiphol,1.243261
4,MUC,Munich International Airport,1.230537
5,ORD,Chicago O'Hare International Airport,1.220283
6,DFW,Dallas/Fort Worth International Airport,1.208827
7,DXB,Dubai International Airport,1.208430
8,PEK,Beijing Capital International Airport,1.208074
9,ATL,Hartsfield - Jackson Atlanta International Airport,1.199858


The following example takes a long time to run since it calculates the betweenness centrality.
Algorithms for calculating the betweenness centrality have high complexity.

In [85]:
centrality[code, score] <~ BetweennessCentrality(*route[a, b])
?[code, desc, score] := centrality[code, score], *airport{code, desc}

:limit 10;
:order -score

,code,desc,score
0,ANC,Anchorage Ted Stevens,1074869.260952
1,KEF,"Reykjavik, Keflavik International Airport",928449.975037
2,HEL,Helsinki Ventaa,581588.490562
3,PEK,Beijing Capital International Airport,532020.425300
4,DEL,Indira Gandhi International Airport,472979.963291
5,IST,Istanbul International Airport,457882.076744
6,PKC,Yelizovo Airport,408571.027619
7,MSP,Minneapolis-St.Paul International Airport,396433.049206
8,LAX,Los Angeles International Airport,393310.114286
9,DEN,Denver International Airport,374339.835975


These are the airports that, if disconnected from the network, cause the most disruption.
As this example shows, some of the algorithms really struggle when you go beyond small or medium sized dataset.

Community detection can collapse a graph into a _supergraph_.
Here we store the result, since it has too many rows to display nicely:

In [86]:
community[detailed_cluster, code] <~ CommunityDetectionLouvain(*route[a, b])
?[code, cluster, detailed_cluster] := community[detailed_cluster, code], cluster = first(detailed_cluster)

:replace community {code => cluster, detailed_cluster}

,status
0,OK


We can look at the supernodes containing specific nodes. For example, the supernode for London Gatwick consists of mainly UK and other European airports, as you would expect:

In [87]:
community[code] := *community{code: 'LGW', cluster}, *community{code, cluster}
?[country, count(code)] := 
    community[code], 
    *airport{code, desc, country: country_code}, 
    *country{code: country_code, desc: country},

:order -count(code)
:limit 5

,country,count(code)
0,United Kingdom,54
1,France,50
2,Norway,49
3,Spain,40
4,Greece,38


For `JFK` on the other hand, its supernode consists of mainly US airports:

In [88]:
community[code] := *community{code: 'JFK', cluster}, *community{code, cluster}
?[country, count(code)] := 
    community[code], 
    *airport{code, desc, country: country_code}, 
    *country{code: country_code, desc: country},

:order -count(code)
:limit 5

,country,count(code)
0,United States,444
1,Canada,111
2,Brazil,108
3,Mexico,57
4,Colombia,50


But it does not always work according to geography. For example, Frankfurt airport is in Germany:

In [89]:
?[desc, country_desc] := *airport{code: 'FRA', desc, country: country_code}, *country{code: country_code, desc: country_desc}

,desc,country_desc
0,Frankfurt am Main,Germany


But its supernode:

In [90]:
community[code] := *community{code: 'FRA', cluster}, *community{code, cluster}
?[country, count(code)] := 
    community[code], 
    *airport{code, desc, country: country_code}, 
    *country{code: country_code, desc: country},

:order -count(code)
:limit 5

,country,count(code)
0,United States,444
1,Canada,111
2,Brazil,108
3,Mexico,57
4,Colombia,50


Germany does not even appear in the top five. In fact, `FRA` is in the same supernode as `JFK`. What matters is the connectivity in the graph, not the geography.
As another example:

In [91]:
community[code] := *community{code: 'SIN', cluster}, *community{code, cluster}
?[country, count(code)] := 
    community[code], 
    *airport{code, desc, country: country_code}, 
    *country{code: country_code, desc: country},

:order -count(code)
:limit 5

,country,count(code)
0,China,216
1,Australia,125
2,Indonesia,68
3,Japan,65
4,Philippines,40


You'd expect `SIN` to be a Chinese airport. Wrong:

In [92]:
?[desc, country_desc] := *airport{code: 'SIN', desc, country: country_code}, *country{code: country_code, desc: country_desc}

,desc,country_desc
0,"Singapore, Changi International Airport",Singapore


Finally, let's collapse the `route` relation into `super_route`:

In [93]:
?[fr_cluster, to_cluster, count(dist), sum(dist)] := *route{fr, to, dist}, 
                                                     *community{code: fr, cluster: fr_cluster},
                                                     *community{code: to, cluster: to_cluster}
:replace super_route {fr_cluster, to_cluster => n_routes=count(dist), total_distance=sum(dist)}

,status
0,OK


As expected, the "diagonals" where `fr_cluster == to_cluster` are larger in the `super_route` graph:

In [94]:
?[fr_cluster, to_cluster, n_routes, total_distance] := *super_route{fr_cluster, to_cluster, n_routes, total_distance}, fr_cluster < 2

,fr_cluster,to_cluster,n_routes,total_distance
0,0,0,9041,8933554.000000
1,0,1,434,1695379.000000
2,0,2,228,761661.000000
3,0,3,530,1681865.000000
4,0,4,163,391892.000000
5,0,8,3,300.000000
6,0,11,2,283.000000
7,0,19,1,238.000000
8,0,21,2,705.000000
9,0,22,1,975.000000


Now the super graph is small enough that all analytics algorithms return instantly:

In [95]:
?[cluster, score] <~ PageRank(*super_route[])
:order -score
:limit 5

,cluster,score
0,3,2.082531
1,0,1.526858
2,2,1.262160
3,1,1.262160
4,4,1.188595


You can now go on to investigate the supernodes, give real-world interpretations to them, etc. 
For example, a naïve interpretation of the above PageRank result is that 
North America is (still) the most prosperous part of the world,
followed by East Asia in the second place, Europe in the third place, and South Asia in the fourth place.

That's it for the tutorial. Now, continue learning with the [Manual](https://cozodb.github.io/current/manual/index.html)!